In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:

df_votants = pd.read_csv('database_deputes.csv', index_col=0)
df_votes2 = pd.read_csv('database_votes2.csv', index_col=0)

In [3]:
df_votes = df_votes2[['idScrutin', 'idVotant']]

In [4]:
copy = df_votes2.copy()
copy = copy.pivot(index = 'idVotant', columns = 'idScrutin', values = 'vote')

In [5]:
#on remplace les NaN par de l'abstention (en confondant abstention et non-votant)
copy = copy.fillna('0')

copy = copy.replace(['Pour'],'1')
copy = copy.replace(['Contre'],'1')
copy = copy.replace(['Non-votant'],'1')

In [6]:
copy2=copy.unstack()

In [7]:
copy2 = copy2.reset_index()
copy2 = copy2.rename(columns = {0 : 'présence'})
copy2

,idScrutin,idVotant,présence
0,0,PA1008,0
1,0,PA1012,0
2,0,PA1029,0
3,0,PA1198,0
4,0,PA1206,0
...,...,...,...
1792270,3116,PA774962,0
1792271,3116,PA856,0
1792272,3116,PA923,0
1792273,3116,PA942,0


In [8]:
copy2 = copy2.sort_values(['idScrutin', 'idVotant'])
copy2


,idScrutin,idVotant,présence
0,0,PA1008,0
1,0,PA1012,0
2,0,PA1029,0
3,0,PA1198,0
4,0,PA1206,0
...,...,...,...
1792270,3116,PA774962,0
1792271,3116,PA856,0
1792272,3116,PA923,0
1792273,3116,PA942,0


In [9]:
bjr = df_votes2.merge(copy2, on = ['idScrutin', 'idVotant'], how = 'outer')
bjr

,idScrutin,idVotant,vote,organeRefGroupe,date_scrutin,code_type_vote,titre,demandeur,organe_ref,votants,...,Département,Numéro de circonscription,Profession,Groupe politique (complet),Groupe politique (abrégé),date_naissance,type_mandat,type_organe,qualite_mandat,présence
0,0,PA606171,Non-votant,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57.0,...,Finistère,6.0,Cadre du secteur privé,La République en Marche,LaREM,1962-07-01,MandatParlementaire_type,ASSEMBLEE,membre,1
1,0,PA721824,Non-votant,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57.0,...,Paris,13.0,Cadre supérieur (entreprises publiques),La République en Marche,LaREM,NaN,NaN,NaN,NaN,1
2,0,PA719952,Pour,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57.0,...,Loire,2.0,Autre cadre (secteur privé),La République en Marche,LaREM,NaN,NaN,NaN,NaN,1
3,0,PA607395,Pour,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57.0,...,Manche,3.0,Cadre commercial,La République en Marche,LaREM,1969-10-12,MandatParlementaire_type,ASSEMBLEE,membre,1
4,0,PA718710,Pour,PO730964,2020-10-07,SPO,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",PO717460,57.0,...,Hautes-Alpes,1.0,NaN,La République en Marche,LaREM,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792270,3116,PA774962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792271,3116,PA856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792272,3116,PA923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792273,3116,PA942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [10]:
df_votants = pd.read_csv('database_votants.csv', index_col=0)

C:\Users\Clara\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3,4,5,6,7,8,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
df_votants2=df_votes2.drop_duplicates('idVotant')[['idVotant','Département','Numéro de circonscription','Profession','Groupe politique (complet)','Groupe politique (abrégé)','date_naissance']]

In [12]:
bjr=df_votants2.merge(bjr, on='idVotant',how='outer')

In [13]:
df_scrutins2 = df_votes2.drop_duplicates('idScrutin')[['idScrutin','titre','demandeur','date_scrutin']]

In [14]:
bjr=df_scrutins2.merge(bjr, on='idScrutin',how='outer')
bjr

,idScrutin,titre_x,demandeur_x,date_scrutin_x,idVotant,Département_x,Numéro de circonscription_x,Profession_x,Groupe politique (complet)_x,Groupe politique (abrégé)_x,...,Département_y,Numéro de circonscription_y,Profession_y,Groupe politique (complet)_y,Groupe politique (abrégé)_y,date_naissance_y,type_mandat,type_organe,qualite_mandat,présence
0,0,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",2020-10-07,PA606171,Finistère,6,Cadre du secteur privé,La République en Marche,LaREM,...,Finistère,6.0,Cadre du secteur privé,La République en Marche,LaREM,1962-07-01,MandatParlementaire_type,ASSEMBLEE,membre,1
1,0,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",2020-10-07,PA721824,Paris,13,Cadre supérieur (entreprises publiques),La République en Marche,LaREM,...,Paris,13.0,Cadre supérieur (entreprises publiques),La République en Marche,LaREM,NaN,NaN,NaN,NaN,1
2,0,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",2020-10-07,PA719952,Loire,2,Autre cadre (secteur privé),La République en Marche,LaREM,...,Loire,2.0,Autre cadre (secteur privé),La République en Marche,LaREM,NaN,NaN,NaN,NaN,1
3,0,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",2020-10-07,PA607395,Manche,3,Cadre commercial,La République en Marche,LaREM,...,Manche,3.0,Cadre commercial,La République en Marche,LaREM,1969-10-12,MandatParlementaire_type,ASSEMBLEE,membre,1
4,0,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",2020-10-07,PA718710,Hautes-Alpes,1,NaN,La République en Marche,LaREM,...,Hautes-Alpes,1.0,NaN,La République en Marche,LaREM,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792270,3116,l'amendement n° 1237 de la commission des fina...,"Présidente du groupe ""Socialistes et apparentés""",2020-10-30,PA643192,Orne,1,Psychologue,Socialistes et apparentés,SOC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792271,3116,l'amendement n° 1237 de la commission des fina...,"Présidente du groupe ""Socialistes et apparentés""",2020-10-30,PA721768,Paris,12,Cadre en entreprise,La République en Marche,LaREM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792272,3116,l'amendement n° 1237 de la commission des fina...,"Présidente du groupe ""Socialistes et apparentés""",2020-10-30,PA774962,Réunion,2,Professeure des écoles,Gauche démocrate et républicaine,GDR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1792273,3116,l'amendement n° 1237 de la commission des fina...,"Présidente du groupe ""Socialistes et apparentés""",2020-10-30,PA721666,Var,3,Retraitée,Les Républicains,LR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [15]:
df_régions = pd.read_csv('database_deputes.csv', index_col=0)
df_régions = df_régions.drop_duplicates('idVotant')[['idVotant','Région']]
bjr = bjr.merge(df_régions, on ='idVotant', how = 'outer')
df_age = pd.read_csv('age.csv', index_col=0)
bjr = bjr.merge(df_age, on ='idVotant', how = 'outer')
df_accord = pd.read_csv('indice_accord.csv', index_col=0)
bjr = bjr.merge(df_accord, on ='idVotant', how = 'outer')
df_vote = pd.read_csv('proportion_vote.csv', index_col=0)
bjr = bjr.merge(df_vote, on ='idVotant', how = 'outer')
df_genre = pd.read_csv('genre.csv', index_col=0)
bjr = bjr.merge(df_genre, on ='idVotant', how = 'outer')
df_csp = pd.read_excel('csp.xlsx')
bjr = bjr.merge(df_csp, on ='idVotant', how = 'outer')

In [16]:
bjr

,idScrutin,titre_x,demandeur_x,date_scrutin_x,idVotant,Département_x,Numéro de circonscription_x,Profession_x,Groupe politique (complet)_x,Groupe politique (abrégé)_x,...,date_naissance,wikilink,age,IndiceAccord,Parti,Nom_Parti,Contre,Pour,Genre,CSP
0,0,l'article 24 bis du projet de loi portant dive...,"Président du groupe ""La République en Marche""",2020-10-07,PA606171,Finistère,6,Cadre du secteur privé,La République en Marche,LaREM,...,1962-07-01,Richard_Ferrand,58,24.43418,PO730964,LREM,5.034642,70.161663,M.,Cadres d'entreprise
1,1,l'amendement n°23 de M. Potier à l'article 62 ...,"Présidente du groupe ""Socialistes et apparentés""",2019-03-15,PA606171,Finistère,6,Cadre du secteur privé,La République en Marche,LaREM,...,1962-07-01,Richard_Ferrand,58,24.43418,PO730964,LREM,5.034642,70.161663,M.,Cadres d'entreprise
2,2,"la motion de rejet préalable, déposée par Mme ...","Présidente du groupe ""Socialistes et apparentés""",2019-11-13,PA606171,Finistère,6,Cadre du secteur privé,La République en Marche,LaREM,...,1962-07-01,Richard_Ferrand,58,24.43418,PO730964,LREM,5.034642,70.161663,M.,Cadres d'entreprise
3,3,l'amendement n° 400 de Mme Louwagie à l'articl...,"Président du groupe ""Les Républicains""",2017-10-19,PA606171,Finistère,6,Cadre du secteur privé,La République en Marche,LaREM,...,1962-07-01,Richard_Ferrand,58,24.43418,PO730964,LREM,5.034642,70.161663,M.,Cadres d'entreprise
4,4,l'amendement n° 272 de Mme Obono à l'article 5...,"Président du groupe ""La France insoumise""",2018-12-04,PA606171,Finistère,6,Cadre du secteur privé,La République en Marche,LaREM,...,1962-07-01,Richard_Ferrand,58,24.43418,PO730964,LREM,5.034642,70.161663,M.,Cadres d'entreprise
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1792270,3112,l'amendement n° 885 de M. Nilor après l'articl...,"Président du groupe ""de la Gauche démocrate et...",2017-11-09,PA720802,Pyrénées-Orientales,2,non communiqué,Non inscrit,NI,...,NaN,Catherine_Pujol_(femme_politique),60,36.00000,PO723569,Non inscrits,32.000000,68.000000,Mme,Sans profession dÃ©clarÃ©e
1792271,3113,l'amendement n° 186 de M. Descoeur et les amen...,"Président du groupe ""Libertés et Territoires""",2019-01-23,PA720802,Pyrénées-Orientales,2,non communiqué,Non inscrit,NI,...,NaN,Catherine_Pujol_(femme_politique),60,36.00000,PO723569,Non inscrits,32.000000,68.000000,Mme,Sans profession dÃ©clarÃ©e
1792272,3114,l'amendement n° 991 de M. Prud'homme à l'artic...,"Président du groupe ""La France insoumise""",2018-04-22,PA720802,Pyrénées-Orientales,2,non communiqué,Non inscrit,NI,...,NaN,Catherine_Pujol_(femme_politique),60,36.00000,PO723569,Non inscrits,32.000000,68.000000,Mme,Sans profession dÃ©clarÃ©e
1792273,3115,l'article unique du projet de loi autorisant l...,"Président du groupe ""La France insoumise""",2017-12-18,PA720802,Pyrénées-Orientales,2,non communiqué,Non inscrit,NI,...,NaN,Catherine_Pujol_(femme_politique),60,36.00000,PO723569,Non inscrits,32.000000,68.000000,Mme,Sans profession dÃ©clarÃ©e


In [17]:
df_randomforest = pd.read_csv('df_randomforest.csv', index_col=0,low_memory=False)
df_randomforest = df_randomforest.drop_duplicates('idScrutin')[['idScrutin','ratioabstention','cluster']]
df_randomforest = df_randomforest.merge(bjr, on ='idScrutin', how='outer')

In [18]:
df_randomforest.columns

Index(['idScrutin', 'ratioabstention', 'cluster', 'titre_x', 'demandeur_x',
       'date_scrutin_x', 'idVotant', 'Département_x',
       'Numéro de circonscription_x', 'Profession_x',
       'Groupe politique (complet)_x', 'Groupe politique (abrégé)_x',
       'date_naissance_x', 'vote', 'organeRefGroupe', 'date_scrutin_y',
       'code_type_vote', 'titre_y', 'demandeur_y', 'organe_ref', 'votants',
       'pour', 'contre', 'non_votants', 'non_votants_volontaires',
       'abstention', 'resultat', 'Prénom_x', 'Nom_x', 'Région_x',
       'Département_y', 'Numéro de circonscription_y', 'Profession_y',
       'Groupe politique (complet)_y', 'Groupe politique (abrégé)_y',
       'date_naissance_y', 'type_mandat', 'type_organe', 'qualite_mandat',
       'présence', 'Région_y', 'Nom_y', 'Prénom_y', 'date_naissance',
       'wikilink', 'age', 'IndiceAccord', 'Parti', 'Nom_Parti', 'Contre',
       'Pour', 'Genre', 'CSP'],
      dtype='object')

In [19]:
df_randomforest3=df_randomforest.drop(['organeRefGroupe','idScrutin','vote','abstention','titre_x','date_scrutin_x','Profession_x','date_naissance_x','Région_x','date_naissance','resultat','date_scrutin_x','demandeur_x','Prénom_x','Nom_x','Groupe politique (abrégé)_x','wikilink','Département_x','Numéro de circonscription_x','code_type_vote','non_votants','type_organe','type_mandat','non_votants_volontaires','qualite_mandat','organe_ref','pour','contre','votants'],axis=1)

In [20]:
df_randomforest3['Région2']=df_randomforest3['Région_y']

In [21]:
df_randomforest3.drop(df_randomforest3.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [22]:
df_randomforest3 = df_randomforest3.drop((['idVotant']),axis=1).reset_index()

In [23]:
df_randomforest_19=df_randomforest3[df_randomforest3['cluster']==33]


In [24]:
df_randomforest_19_présence=df_randomforest_19['présence']

In [25]:
df_randomforest_19.columns

Index(['index', 'ratioabstention', 'cluster', 'Groupe politique (complet)_x',
       'présence', 'age', 'IndiceAccord', 'Parti', 'Nom_Parti', 'Contre',
       'Pour', 'Genre', 'CSP', 'Région2'],
      dtype='object')

In [26]:
df_randomforest_19_présence=df_randomforest_19_présence.astype(int).to_frame()
df_randomforest_19=df_randomforest_19.drop('présence', axis =1)
df_randomforest_19=df_randomforest_19.drop('Groupe politique (complet)_x', axis=1)
df_randomforest_19

,index,ratioabstention,cluster,age,IndiceAccord,Parti,Nom_Parti,Contre,Pour,Genre,CSP,Région2
33350,33350,0.820870,33,58,24.434180,PO730964,LREM,5.034642,70.161663,M.,Cadres d'entreprise,Bretagne
33351,33351,0.820870,33,42,28.533333,PO730964,LREM,19.466667,19.733333,M.,"Cadres de la fonction publique, professions in...",Ile-de-France
33352,33352,0.820870,33,53,74.000000,PO730964,LREM,64.571429,35.428571,M.,"Cadres de la fonction publique, professions in...",Auvergne-Rhône-Alpes
33353,33353,0.820870,33,51,71.171171,PO730964,LREM,60.360360,39.189189,M.,Cadres d'entreprise,Normandie
33354,33354,0.820870,33,55,76.120959,PO730964,LREM,69.968717,30.031283,Mme,Sans profession dÃ©clarÃ©e,Provence-Alpes-Côte d'Azur
...,...,...,...,...,...,...,...,...,...,...,...,...
1689345,1689345,0.914783,33,62,45.161290,PO758835,Socialistes,41.935484,58.064516,Mme,"Cadres de la fonction publique, professions in...",Normandie
1689346,1689346,0.914783,33,42,88.461538,PO730964,LREM,73.076923,26.923077,Mme,Cadres d'entreprise,Ile-de-France
1689347,1689347,0.914783,33,35,40.000000,PO730940,Gauche Dem et Rep,80.000000,20.000000,Mme,"Professions intermÃ©diaires de l'enseignement,...",Réunion
1689348,1689348,0.914783,33,72,58.333333,PO730934,LR,75.000000,25.000000,Mme,Anciens employÃ©s et ouvriers,Provence-Alpes-Côte d'Azur


In [27]:
df_randomforest_19_dummies=pd.get_dummies(df_randomforest_19)

df_randomforest_19_dummies

,index,ratioabstention,cluster,IndiceAccord,Contre,Pour,age_ a,age_27,age_28,age_29,...,Région2_Nouvelle-Aquitaine,Région2_Nouvelle-Calédonie,Région2_Occitanie,Région2_Pays de la Loire,Région2_Polynésie française,Région2_Provence-Alpes-Côte d'Azur,Région2_Réunion,Région2_Saint-Barthélemy et Saint-Martin,Région2_Saint-Pierre-et-Miquelon,Région2_Wallis-et-Futuna
33350,33350,0.820870,33,24.434180,5.034642,70.161663,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33351,33351,0.820870,33,28.533333,19.466667,19.733333,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33352,33352,0.820870,33,74.000000,64.571429,35.428571,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33353,33353,0.820870,33,71.171171,60.360360,39.189189,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33354,33354,0.820870,33,76.120959,69.968717,30.031283,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689345,1689345,0.914783,33,45.161290,41.935484,58.064516,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1689346,1689346,0.914783,33,88.461538,73.076923,26.923077,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1689347,1689347,0.914783,33,40.000000,80.000000,20.000000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1689348,1689348,0.914783,33,58.333333,75.000000,25.000000,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [28]:
df_randomforest_19_dummies= df_randomforest_19_dummies.drop('index',axis=1)


In [29]:
label_19 = df_randomforest_19_présence['présence']
label_19

33350      1
33351      1
33352      0
33353      0
33354      0
          ..
1689345    0
1689346    0
1689347    0
1689348    0
1689349    0
Name: présence, Length: 17825, dtype: int32

In [30]:
from sklearn.model_selection import train_test_split


In [31]:
train_features, test_features, train_labels, test_labels = train_test_split(df_randomforest_19_dummies, label_19, test_size = 0.25)

In [33]:
from sklearn.utils import resample,shuffle
test_label_counts = test_labels.value_counts()
test_features_absent = test_features[test_labels==0]
test_labels_absent = test_labels[test_labels==0]
features_test_less, labels_test_less = resample(test_features_absent,test_labels_absent,n_samples=test_label_counts[1],replace=False)
features_test_ = pd.concat([features_test_less,test_features[test_labels==1]])
labels_test_ = pd.concat([labels_test_less,test_labels[test_labels==1]])
test_features_, test_labels_ = shuffle(features_test_,labels_test_)
print(test_labels_.value_counts())

1    439
0    439
Name: présence, dtype: int64


In [34]:
train_label_counts = train_labels.value_counts()
train_features_absent = (train_features[train_labels==0])
train_labels_absent = (train_labels[train_labels==0])
features_train_less, labels_train_less = resample(train_features_absent,train_labels_absent,n_samples=train_label_counts[1],replace=False)
features_train_ = pd.concat([features_train_less,train_features[train_labels==1]])
labels_train_ = pd.concat([labels_train_less,train_labels[train_labels==1]])
train_features_, train_labels_ = shuffle(features_train_,labels_train_)
print(train_labels_.value_counts())

1    1361
0    1361
Name: présence, dtype: int64


In [70]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth':[5,6,7,8,10], 'n_estimators' : [100,200,300,500,1000]}
predictor= GridSearchCV(RandomForestClassifier(random_state=0),param_grid=param_grid)
predictor.fit(train_features_,train_labels_)
print('Paramètre sélectionné:',predictor.best_params_)
print('Score d\'apprentissage: ',predictor.score(train_features_,train_labels_))
print('Score de test: ',predictor.score(test_features_,test_labels_))

Paramètre sélectionné: {'max_depth': 10, 'n_estimators': 500}
Score d'apprentissage:  0.7817781043350478
Score de test:  0.6719817767653758


In [58]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 100, max_depth=3)

rf.fit(train_features_, train_labels_);

In [59]:
predictions_test = rf.predict(test_features_)

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import recall_score, f1_score

test_acc = accuracy_score(test_labels_, predictions_test)
print(test_acc)



0.6287015945330297


In [39]:
model_boosting = GradientBoostingClassifier(loss="deviance",
    learning_rate=0.2,
    max_depth=5,
    max_features="sqrt",
    subsample=0.95,
    n_estimators=200)


model_boosting.fit(train_features_, train_labels_)


predictions_test_xgb = model_boosting.predict(test_features_)
predictions_train_xgb = model_boosting.predict(train_features_)


train_acc = accuracy_score(train_labels_, predictions_train_xgb)
print(train_acc)

test_acc = accuracy_score(test_labels_, predictions_test_xgb)
print(test_acc)

0.8802351212343865
0.693621867881549


In [75]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth':[5,6,7,8,10], 'n_estimators' : [300,500,1000]}
predictor= GridSearchCV(GradientBoostingClassifier(random_state=0),param_grid=param_grid)
predictor.fit(train_features_,train_labels_)
print('Paramètre sélectionné:',predictor.best_params_)
print('Score d\'apprentissage: ',predictor.score(train_features_,train_labels_))
print('Score de test: ',predictor.score(test_features_,test_labels_))

Paramètre sélectionné: {'max_depth': 5, 'n_estimators': 500}
Score d'apprentissage:  0.9452608376193975
Score de test:  0.6890660592255126


In [40]:
# Calcul du recall pour Random Forest

recall = recall_score(test_labels_, predictions_test, average='macro')
print('Recall: %.3f' % recall)

# Calcul du recall pour XGBoost

recall = recall_score(test_labels_, predictions_test_xgb, average='macro')
print('Recall: %.3f' % recall)

Recall: 0.637
Recall: 0.694


In [41]:
# Calcul du F1-Score pour Random Forest

f1 = f1_score(test_labels_, predictions_test, average='macro')
print('F1-Score: %.3f' % f1)

# Calcul du F1-Score pour XGBoost

f1 = f1_score(test_labels_, predictions_test_xgb, average='macro')
print('F1-Score: %.3f' % f1)

F1-Score: 0.637
F1-Score: 0.693
